<a href="https://colab.research.google.com/github/arun8166/BCS_tasks/blob/main/task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Brain and Cognitive Society, IIT Kanpur**
## **Introduction to Deep Learning Workshop**
**This python notebook is an assingment on ML/DL**

In this assingment you will solve a **regression** problem of predicting House prices using basic python libraries, and build a **neural network** for handwritten digit identification using **TensorFlow**

## **Linear Regression**
We will use Linear regression for predicting house prices

We are using a Kaggle dataset- https://www.kaggle.com/harlfoxem/housesalesprediction

In [ ]:
# Lets import required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


### **Dataset Preparation**

In [ ]:
#@title
# Execute this cell for loading dataset in a pandas dataframe

from IPython.display import clear_output
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=16x6-8Znn2T50zFwVvKlzsdN7Jd1hpjct' -O Linear_regression_dataset

data_df = pd.read_csv("Linear_regression_dataset")

In [ ]:
print("(No of rows, No of Columns) = ",data_df.shape)
data_df.head()

In [ ]:
# Lets have a quick Look at dataset

print("(No of rows, No of Columns) = ",data_df.shape)
data_df.head()

So there are **19** features (of course we will not use id as feature :) ), and 1 variable to predict(price)

But note that the **date** column contain strings so first we will remove T00.. part from it and than convert it to numpy array.

In [ ]:
data_df['date'] = data_df['date'].str.replace('T000000', '')

data_array=np.array(data_df)
data_array=np.delete(data_array,0,1)
data_array=data_array.astype(np.int)
print(data_array)

assert (data_array.shape == (21613,20))

data_df.head()




Now the next task is **normalization**.

We will scale each column of dataset by x -> (x-u)/s

where u is mean(x), and s is standard deviation of u

In [ ]:

mean_array = np.mean(data_array,0)                                        # this should be an array, each entry should be mean of a column
sd_array = np.mean(data_array,0)                                    # this should be an array, each entry should be standard deviation of a column

data_array_norm = (data_array - mean_array)/sd_array
print(data_array_norm)

print(data_array_norm.shape)

The last step is to make train and test dataset and to create seperate vector for price

In [ ]:
labels = data_array_norm[:,1]                                                                                                            # extract the price column from data

x_array_norm = np.delete(data_array_norm,1,1)                                                                                                      # delete the price column from data_array_norm. Hint: use np.delete()

x_train, x_test, y_train, y_test = train_test_split(x_array_norm,labels,test_size=0.15,random_state=42,shuffle=True)    # splitting data into test and train set.

print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

### **Loss and gradient descent**
We will use mean squared error(MSE) as loss

Use the gradient descent algorithm which you learned from tutorials

Your task is to complete the following functions

In [ ]:
def loss(y_pred,y_true):

  mse = np.mean(np.square(np.subtract(y_true,y_pred)))     #mse = mean_squared_error(y_true,y_pred)


  return mse

In [ ]:
def y(x,a,b):
  m,n = x.shape
  
  y_pred = np.dot(x,a) + b
  y_pred = y_pred.astype(np.float)

  #assert(y_pred.shape == (m,))
  return y_pred

In [ ]:
def gradient(x,a,b,y_true):
  """
  This function shoud return gradient of loss
  input:
  x: [array] the feature vector of shape (m,n)
  a: [array] weights of shape (n,)
  b: [scalar] bias
  y_true: [array] ground truth of shape (m,)

  output:
  grad: [tuple] a tuple (derivative with respect to a[array of shape(n,)], derivative with respect to b[scalar])
  """
  m,n = x.shape
  yp = y(x,a,b)
  y_true = y_true.astype(np.float)
  d=[1]*n
  for k in range(0,n,1):
    d[k] = 2*np.mean(x[:,k]*np.subtract(yp,y_true))
  da = np.array(d)                                   # write code to calculate derivative of loss with respect to a
  db = 2*np.mean(np.subtract(yp,y_true))             # write code to calculate derivative of loss with respect to b

  assert(da.shape ==(n,))
  return (da,db)

In [ ]:
def gradient_descent(x,y_true,learning_rate=0.01,epochs = 100):
  """
  This function perfroms gradient descent and minimizes loss
  input:
  x: [array] the feature vector of shape (m,n)
  y_true: [array] ground truth of shape (m,)
  
  output:
  loss: [array] of size (epochs,)
  weights: [tuple] (a,b)
  """
  m,n = x.shape
  loss_mse = []
  v = [0]*n                                 # initialize empty list to store loss
  a = np.array(v) 
  n, = a.shape                              # initialize a- weights and b- bias
  b = 0 

  for i in range(epochs):
    # calculate derivative using gradient() function
    # apply gradient descent now to update a and b
    for j in range(0,n,1):
      a[j] = a[j] - learning_rate*gradient(x,a,b,y_true)[0][j]
    b = b - learning_rate*gradient(x,a,b,y_true)[1]

    l_mse = loss(y(x,a,b),y_true)                                # calculate loss at this point
    loss_mse.append(l_mse)

    print("Epoch ",i+1," Completed!","loss = ",l_mse)
  
  print("Training completed!!")

  assert(a.shape==(n,))

  return (loss_mse,a,b)

### **Training** 

In [ ]:
epochs = 150             # tweak this!!!
learn_rate = 0.01     # choose learning rate wisely otherwise loss may diverge!!

train_loss,a,b = gradient_descent(x_train,y_train,learn_rate,epochs)

### **Evaluation and Visualization**
Lets plot how loss varies with epochs


In [ ]:
test_loss = gradient_descent(x_test,y_test,learn_rate,epochs)
print("Loss on test data = ",test_loss)
epo=np.linspace(0,epochs,epochs)

plt.plot(epo,test_loss[0])                   # plot loss versus epochs
plt.title("Training Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()



## **Deep Learning**
In this section We will build a simple multilayer perceptron network(**MLP**) in TensorFlow

In [ ]:
# Lets import the required libraries
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

### **Load Dataset**
We will be using MNIST dataset of handwritten digits

Just run the cell below to load dataset

In [ ]:
mnist = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print("No. of training examples = ",x_train.shape[0])
print("Size of each image in dataset = ",x_train.shape[1:])
print("No. of test examples = ",x_test.shape[0])

In [ ]:
# Run this cell to visualize some of the images from dataset

n = 10    # = no. of images to visualize

index = np.random.choice(x_train.shape[0],10)  # choose random index
print("label: ",end="")

for i,ind in enumerate(index):
    plt.subplot(1,n,i+1)
    plt.imshow(x_train[ind],cmap="gray")
    plt.axis("off")
    print(y_train[ind],end="       ")

plt.show()


#### Preprocess dataset
Since we are building a MLP model the input to the model should be a vector rather than a 28 by 28 matrix.

So your **First Task** is to flatten the images

(Hint: use *reshape()* method of arrays...)

Next, create validation dataset out of training dataset.

You can use 50K images for training and 10K for validation

In [ ]:
# Flatten the images into 1-d vectors
x_train = x_train/256
x_test = x_test/256

x_train_flatten = x_train.reshape(60000,784)  
x_test                                     # flatten the images of training set
x_test_flatten = x_test.reshape(10000,784)                                        # flatten th eimages of test set


# Divide the training data into training and validation data....

n_validation = 10000 
x_train, x_validation, y_train, y_validation = train_test_split(x_train,y_train,test_size=1/6,random_state=42,shuffle=True)
x_validation = x_validation.reshape(10000,784)
x_train_flatten = x_train.reshape(50000,784)                                      # choose number of images to be used for validation
print(x_train.shape, y_train.shape)


### **Build a model**
You can choose whatever architechure you want, but ensure that it is **not too deep** as that will take too much time to train and **not too shallow** as that will give very low accuracy.

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(10, input_shape=(784,), activation='sigmoid')]
    )

# Make a graphical representation of the model...
keras.utils.plot_model(model,show_shapes=True)
model.summary

#### Compile and Train
Choose an optimizer- method that minimizes loss function

**adam** optimizer is one of the popular choices. You should read about these online

In [ ]:
from keras import losses, optimizers, metrics
model.compile(optimizer = "adam", loss = 'sparse_categorical_crossentropy',metrics=["accuracy"])

n_epochs = 20              # set number of epochs
batch_size = 512            # you can tweak with these parametrs
history = model.fit(x_train_flatten,y_train,epochs=20,validation_data=(x_validation,y_validation))
print(history.history.keys())

### **Evaluate**
Evaluate your model on test data.

And Show some results

In [ ]:
results = model.evaluate(x_test_flatten,y_test, 20)
print("Loss = ",results[0])
print("Accuracy = ",results[1]*100,"%")

# Plot Accuracy...
plt.plot([i for i in range(n_epochs)],history.history['accuracy'], label="Training accuracy")
plt.plot([i for i in range(n_epochs)],history.history['val_accuracy'],label="validation accuracy" )
plt.title("Model accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

# Similarly write code to plot loss...
plt.plot([i for i in range(n_epochs)],history.history['loss'], label="loss")
plt.title("Model loss")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()



Lets show our results on images from testing dataset

In [ ]:
n = 10   # = no. of images to see predictions on
index = np.random.choice(x_test.shape[0],10)  # choose random index from test data
print("label: ",end="")

for i,ind in enumerate(index):
    plt.subplot(1,n,i+1)
    plt.imshow(x_test[ind],cmap="gray")             # fill code to show images from test set
    plt.axis("off")
    print(y_test[ind],end="       ")
print(index)
    
    
    



plt.show()
print("Predicted value: ")

# Now lets print the predictions

for i,ind in enumerate(index):
    # write code to predict and print digit in image
    # Hint: the output of the model is a 10-d vector which gives probabilties
    # The digit in the image would be the class for which probability is hghest..
    di = model.predict(x_test_flatten)
    digit = np.argmax(di[ind])

    #digits = model.predict(x_test_flatten)
    print(digit,end="       ")
    

That's it you have completed the assignment !!

We hope that you learned something from this exercise

### Credits:

**Leaders:**

Mohit Kulkarni

Shivanshu Tyagi

**Scretaries:**

Sahil Bansal

Shashwat Gupta

Rashmi Sharma